# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
Training = pd.read_csv('Dataset/train_clean.csv')
Testing = pd.read_csv('Dataset/test_clean.csv')

In [25]:
Training.head()

,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Region_Asia,Region_Central America and Caribbean,Region_European Union,Region_Middle East,Region_North America,Region_Oceania,Region_Rest of Europe,Region_South America,Life_expectancy
0,-1.229155,1.347790,1.444716,1.667362,-0.984910,-0.322375,-0.742607,-1.299421,-2.292760,-2.320385,2.088477,-1.529682,-0.313071,1.098618,1.058457,-1.122641,0,0,0,0,0,0,0,0,0,53.7
1,1.586825,-1.301510,-1.215595,-0.959561,-0.043598,0.999495,1.156882,0.254490,0.906471,0.918252,-0.376160,0.354779,0.154691,-0.285022,-0.337966,1.169452,0,0,1,0,0,0,0,0,0,78.6
2,-1.445769,-1.339502,-1.273397,-1.037472,1.784384,-0.175500,0.776984,0.483006,0.582498,0.672280,-0.499933,1.160137,0.987709,-1.014578,-1.036178,0.258895,1,0,0,1,0,0,0,0,0,79.4
3,0.936983,0.504701,0.519060,2.370735,0.594705,-1.277058,-1.339590,0.940039,-1.523325,-1.869436,4.529512,0.209314,1.159193,0.343905,0.659479,0.667076,0,0,0,0,0,0,0,0,0,60.1
4,-1.445769,0.271813,0.150413,0.138674,-0.031180,0.044811,0.125731,0.437303,0.258525,0.262326,0.062099,0.606148,-0.997520,0.494848,0.609607,0.510083,0,0,1,0,0,0,0,0,0,69.7


# Prediction Function

In [19]:
def predict(x, w, b):
    return np.dot(x, w) + b

# Cost Function

In [23]:
def cost_function(x, w, b, y):
    m = len(x)
    y_hat = predict(x, w, b)
    return (1 / (m)) * np.sum(np.pow((y_hat - y), 2))

# Initialize the Parameters

In [20]:
w, b = 0.0, 0.0

# Gradient Derivative

In [12]:
def gradient_derivatives(x, w, b, y):
    m = len(x)
    y_hat = predict(x, w, b)
    dw = (1 / m) * np.sum((y_hat - y) * x)
    db = (1 / m) * np.sum(y_hat - y)
    return dw, db

# Gradient Descent

In [13]:
def gradient_descent(x, w, b, y, lr, num_iters):

    cost_history = []
    for i in range(num_iters):
        dw, db = gradient_derivatives(x, w, b, y)
        w -= lr * dw
        b -= lr * db

        cost = cost_function(x, w, b, y)
        cost_history.append(cost)
        print(f"Cost at iteration {i} = {cost}")

    return w, b, cost_history


In [14]:
print(w, b)

0.0 0.0


# Train the Model

In [26]:
# ==========================================
# 1. HELPER FUNCTIONS (From Scratch)
# ==========================================
def compute_r2_score(y_true, y_pred):
    # Sum of Squared Residuals (Errors)
    ss_res = np.sum((y_true - y_pred) ** 2)
    # Total Sum of Squares (Variance from the mean)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    
    # Avoid division by zero if variance is 0
    if ss_tot == 0:
        return 0.0
        
    return 1 - (ss_res / ss_tot)

def compute_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# ==========================================
# 2. SETUP DATA
# ==========================================
target_col = "Life_expectancy"

# Ensure inputs are numpy arrays
y_train = Training[target_col].values
y_test = Testing[target_col].values

# Get list of feature names (excluding the target)
feature_columns = [col for col in Training.columns if col != target_col]

# Hyperparameters
lr = 0.01
num_iters = 2000

results_list = []

print(f"Evaluating {len(feature_columns)} features on Training & Testing sets...")

# ==========================================
# 3. THE LOOP (Train -> Test -> Record)
# ==========================================
for feature_name in feature_columns:
    
    # --- A. PREPARE DATA ---
    X_train_feat = Training[feature_name].values
    X_test_feat = Testing[feature_name].values
    
    # --- B. TRAIN (Gradient Descent) ---
    # Always reset weights to 0 for unbiased comparison
    w_init = 0.0
    b_init = 0.0
    
    w_final, b_final, cost_history = gradient_descent(
        X_train_feat, w_init, b_init, y_train, lr, num_iters
    )
    
    # --- C. TEST (Prediction & Metrics) ---
    # Predict on the unseen TESTING data
    y_pred_test = predict(X_test_feat, w_final, b_final)
    
    # Calculate Metrics
    test_mse = compute_mse(y_test, y_pred_test)
    test_r2 = compute_r2_score(y_test, y_pred_test)
    
    # --- D. SAVE RESULTS ---
    results_list.append({
        "Feature": feature_name,
        "Weight_w": w_final,
        "Bias_b": b_final,
        "Train_MSE_Cost": cost_history[-1], # Final error on training
        "Test_MSE_Cost": test_mse,          # Final error on testing
        "Test_R2_Score": test_r2            # "Accuracy" (1.0 is perfect)
    })

# ==========================================
# 4. SAVE TO CSV
# ==========================================
results_df = pd.DataFrame(results_list)

# Sort by R2 Score (Best predictors at the top)
results_df = results_df.sort_values(by="Test_R2_Score", ascending=False)

filename = "feature_performance_metrics.csv"
results_df.to_csv(filename, index=False)

print(f"Success! Analysis saved to '{filename}'.")
print(results_df.head(10)) # Show the top 10 best features

Evaluating 25 features on Training & Testing sets...
Cost at iteration 0 = 4826.926800270663
Cost at iteration 1 = 4732.308050275354
Cost at iteration 2 = 4639.572213246275
Cost at iteration 3 = 4548.6818192185565
Cost at iteration 4 = 4459.600143879562
Cost at iteration 5 = 4372.291193730419
Cost at iteration 6 = 4286.719691542822
Cost at iteration 7 = 4202.851062105246
Cost at iteration 8 = 4120.651418252823
Cost at iteration 9 = 4040.0875471752047
Cost at iteration 10 = 3961.1268969969155
Cost at iteration 11 = 3883.7375636247457
Cost at iteration 12 = 3807.8882778568877
Cost at iteration 13 = 3733.5483927485966
Cost at iteration 14 = 3660.6878712292796
Cost at iteration 15 = 3589.277273965995
Cost at iteration 16 = 3519.287747468478
Cost at iteration 17 = 3450.6910124308724
Cost at iteration 18 = 3383.4593523054614
Cost at iteration 19 = 3317.5656021037807
Cost at iteration 20 = 3252.9831374205905
Cost at iteration 21 = 3189.685863676272
Cost at iteration 22 = 3127.6482055732963
Co

In [22]:
# ==========================================
# 1. SETUP DATA
# ==========================================
target_col = "Life_expectancy"
feature_columns = [col for col in Training.columns if col != target_col]

# Define Targets (Y)
# We don't need to reshape Y, a simple array is fine
y_train = Training[target_col].values
y_test = Testing[target_col].values

results_list = []

print(f"Evaluating {len(feature_columns)} features using Scikit-Learn...")

# ==========================================
# 2. THE LOOP (Train -> Test -> Record)
# ==========================================
for feature_name in feature_columns:
    
    # --- A. PREPARE DATA (Reshaping is Critical for Sklearn) ---
    # Sklearn expects a 2D array for features: [[Value1], [Value2], ...]
    # .reshape(-1, 1) converts a flat row into a column
    X_train_feat = Training[feature_name].values.reshape(-1, 1)
    X_test_feat = Testing[feature_name].values.reshape(-1, 1)
    
    # --- B. TRAIN (Linear Regression) ---
    # Initialize the built-in model
    model = LinearRegression()
    
    # Fit on the training data
    model.fit(X_train_feat, y_train)
    
    # Get the "scratch" equivalents
    w_final = model.coef_[0]   # The Weight (Slope)
    b_final = model.intercept_ # The Bias (Intercept)
    
    # --- C. TEST (Prediction & Metrics) ---
    # Predict on unseen TESTING data
    y_pred_train = model.predict(X_train_feat) # To calculate Train MSE
    y_pred_test = model.predict(X_test_feat)   # To calculate Test Metrics
    
    # Calculate Built-in Metrics
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # --- D. SAVE RESULTS ---
    results_list.append({
        "Feature": feature_name,
        "Weight_w": w_final,
        "Bias_b": b_final,
        "Train_MSE_Cost": train_mse,
        "Test_MSE_Cost": test_mse,
        "Test_R2_Score": test_r2
    })

# ==========================================
# 3. SAVE TO CSV
# ==========================================
results_df = pd.DataFrame(results_list)

# Sort by R2 Score to match your previous view
results_df = results_df.sort_values(by="Test_R2_Score", ascending=False)

filename = "feature_performance_sklearn.csv"
results_df.to_csv(filename, index=False)

print(f"Success! Analysis saved to '{filename}'.")
print(results_df.head(10))

Evaluating 25 features using Scikit-Learn...
Success! Analysis saved to 'feature_performance_sklearn.csv'.
                     Feature   Weight_w     Bias_b  Train_MSE_Cost  \
3            Adult_mortality  -8.134295  69.644444        6.981427   
2          Under_five_deaths  -7.971391  69.644444        9.603910   
1              Infant_deaths  -7.802929  69.644444       12.260073   
11            GDP_per_capita   7.000486  69.644444       24.133559   
15                 Schooling   6.510087  69.644444       30.756122   
8                      Polio   5.671774  69.644444       40.963679   
9                 Diphtheria   5.628932  69.644444       41.447601   
7                        BMI   5.216563  69.644444       45.917916   
10             Incidents_HIV  -5.318011  69.644444       44.849691   
16  Economy_status_Developed  11.286792  67.239062       51.754868   

    Test_MSE_Cost  Test_R2_Score  
3        6.937173       0.889147  
2        8.681617       0.861272  
1       10.955352

In [27]:
# 1. Load the two result files
manual_df = pd.read_csv("feature_performance_metrics.csv")
sklearn_df = pd.read_csv("feature_performance_sklearn.csv")

# Set 'Feature' as index for easier lookup
manual_df.set_index("Feature", inplace=True)
sklearn_df.set_index("Feature", inplace=True)

# 2. Loop through every feature found in your results
print("Generating plots...")

for feature_name in manual_df.index:
    # Check if feature exists in the Training data (Safety check)
    if feature_name in Training.columns:
        
        # --- A. PREPARE DATA ---
        X = Training[feature_name].values
        y = Training["Life_expectancy"].values
        
        # --- B. GET WEIGHTS & BIAS ---
        # Manual Model
        w_man = manual_df.loc[feature_name, "Weight_w"]
        b_man = manual_df.loc[feature_name, "Bias_b"]
        r2_man = manual_df.loc[feature_name, "Test_R2_Score"]
        
        # Sklearn Model
        w_sk = sklearn_df.loc[feature_name, "Weight_w"]
        b_sk = sklearn_df.loc[feature_name, "Bias_b"]
        r2_sk = sklearn_df.loc[feature_name, "Test_R2_Score"]
        
        # --- C. CALCULATE LINES ---
        # Create a range of X values (min to max) to draw a smooth line
        x_line = np.linspace(X.min(), X.max(), 100)
        
        y_line_man = w_man * x_line + b_man
        y_line_sk = w_sk * x_line + b_sk
        
        # --- D. PLOT ---
        plt.figure(figsize=(10, 6))
        
        # 1. Scatter Plot of Actual Data
        plt.scatter(X, y, color='lightgray', label='Training Data', alpha=0.6)
        
        # 2. Manual Regression Line (Blue Solid)
        plt.plot(x_line, y_line_man, color='blue', linewidth=3, alpha=0.7, 
                 label=f'Manual Model (R²={r2_man:.2f})')
        
        # 3. Sklearn Regression Line (Red Dashed)
        # We make this dashed so you can see if it sits perfectly on top of the blue line
        plt.plot(x_line, y_line_sk, color='red', linestyle='--', linewidth=2, 
                 label=f'Sklearn Model (R²={r2_sk:.2f})')
        
        # Formatting
        plt.title(f"Regression Comparison: {feature_name}", fontsize=14)
        plt.xlabel(feature_name, fontsize=12)
        plt.ylabel("Life Expectancy", fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # --- E. SAVE ---
        # Clean filename (remove weird characters if any)
        clean_name = feature_name.replace("/", "_").replace(" ", "_")
        plt.savefig(f"plot_{clean_name}.png")
        plt.close() # Close memory to prevent crashing on many loops

print("Success! Plots have been saved for all features.")

Generating plots...
Success! Plots have been saved for all features.
